In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/AlexeyAB/darknet.git
%cd darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15412, done.
remote: Total 15412 (delta 0), reused 0 (delta 0), pack-reused 15412
Receiving objects: 100% (15412/15412), 14.02 MiB | 3.32 MiB/s, done.
Resolving deltas: 100% (10356/10356), done.
/content/darknet


In [3]:
!pwd
!sed -i "s/GPU=0/GPU=1/g" ./Makefile
!sed -i "s/CUDNN=0/CUDNN=1/g" ./Makefile
!sed -i "s/CUDNN_HALF=0/CUDNN_HALF=1/g" ./Makefile
!sed -i "s/OPENCV=0/OPENCV=1/g" ./Makefile
!head Makefile

/content/darknet
GPU=1
CUDNN=1
CUDNN_HALF=1
OPENCV=1
AVX=0
OPENMP=0
LIBSO=0
ZED_CAMERA=0
ZED_CAMERA_v2_8=0



In [4]:
!make
!pwd

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
  

In [ ]:
# !pwd
# %cd ..
# !mkdir Dart_detection
# %cd Dart_detection
# !pwd

/content/darknet
/content
/content/Dart_detection
/content/Dart_detection


In [ ]:
# 複製coco.data/ coco.names去Dart_detection資料夾
# %cd ..
# import os
# import shutil
# if not os.path.exists("Dart_detection"):
#  os.mkdir("Dart_detection")
# if not os.path.exists("Dart_detection/cfg"):
#  os.mkdir("Dart_detection/cfg") 
#  os.mkdir("Dart_detection/cfg/weights")
# if not os.path.exists("Dart_detection/cfg/dart.data"):
#  shutil.copyfile("darknet/cfg/coco.data", "Dart_detection/cfg/dart.data")
# if not os.path.exists("Dart_detection/cfg/face.names"):
#  shutil.copyfile("darknet/cfg/coco.names", "Dart_detection/cfg/dart.names")

/content


In [ ]:
# !pwd
# !cp ./darknet/cfg/yolov4-tiny-custom.cfg ./Dart_detection/cfg/yolov4-tiny-obj.cfg
# %cd Dart_detection/
# !sed -n -e 8p -e 9p -e 212p -e 220p -e 263p -e 269p cfg/yolov4-tiny-obj.cfg

/content
/content/Dart_detection
width=416
height=416
filters=255
classes=80
filters=255
classes=80


In [ ]:
# 更改filters和classes數量
# filters = (C+5)*B
# C為類別數量, B為每個filter可以偵測的BoundingBox數=3, 5為每個BoundingBox輸出5個預測值(x,y,w,h,confidence)
# 我們的case為 C=5, filters=(5+5)*3=30
# !sed -i '212s/255/30/' cfg/yolov4-tiny-obj.cfg  
# !sed -i '220s/80/5/' cfg/yolov4-tiny-obj.cfg
# !sed -i '263s/255/30/' cfg/yolov4-tiny-obj.cfg
# !sed -i '269s/80/5/' cfg/yolov4-tiny-obj.cfg
# # 檢查
# !sed -n -e 212p -e 220p -e 263p -e 269p cfg/yolov4-tiny-obj.cfg

filters=30
classes=5
filters=30
classes=5


In [19]:
# 修改預設 anchors 值
# !pwd
# %cd ..
# %cd /content/darknet
!./darknet detector calc_anchors "/content/drive/MyDrive/Colab_Notebooks/Dart_detection/cfg/dart.data" -num_of_clusters 6 -width 416 -height 416 -showpause

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0

 num_of_clusters = 6, width = 416, height = 416 
 read labels from 15 images 
 loaded 	 image: 15 	 box: 91
 all loaded. 

 calculating k-means++ ...

 iterations = 2 


counters_per_class = 15, 15, 15, 15, 31

 avg IoU = 98.54 % 

Saving anchors to the file: anchors.txt 
anchors =  31, 31,  36, 36,  36, 36,  38, 38,  41, 41,  52, 52



In [ ]:

# 訓練模型
!./darknet detector train \
/content/drive/MyDrive/Colab_Notebooks/Dart_detection/cfg/dart.data \
/content/drive/MyDrive/Colab_Notebooks/Dart_detection/cfg/yolov4-tiny-dart.cfg \
/content/drive/MyDrive/Colab_Notebooks/Dart_detection/backup_weight/yolov4-tiny-dart_last.weights -dont_show

串流輸出內容已截斷至最後 5000 行。
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.509235), count: 6, class_loss = 3.361460, iou_loss = 11.769671, total_loss = 15.131132 
 total_bbox = 157855, rewritten_bbox = 0.961009 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.428228), count: 5, class_loss = 4.295171, iou_loss = 2.683152, total_loss = 6.978323 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.000000), count: 1, class_loss = 0.104026, iou_loss = 0.000000, total_loss = 0.104026 
 total_bbox = 157860, rewritten_bbox = 0.960978 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.472411), count: 5, class_loss = 4.481220, iou_loss = 3.742005, total_loss = 8.223226 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.000000), count: 1, class_loss = 0.130057, iou_loss = 0.000000, total_loss = 0.130057 
 total_bbox = 157865, rewritten_bbo